### Problem Set 8
#### MACS 30150, Dr. Evans
#### Due Monday, Mar. 11 at 11:30am
#### Tianxin Zheng

In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz
from sklearn.model_selection import RandomizedSearchCV, KFold
from scipy.stats import randint as sp_randint
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from scipy.stats import uniform as sp_uniform

#### 1. Decision trees

In [2]:
# load the data
data = pd.read_csv('biden.csv')
data.head()

,biden,female,age,educ,dem,rep
0,90,0,19,12,1,0
1,70,1,51,14,1,0
2,60,0,27,14,0,0
3,50,1,43,14,1,0
4,60,1,38,14,0,1


In [3]:
X = data[['female', 'age', 'educ', 'dem', 'rep']]
y = data.biden

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=25)

In [5]:
biden_tree = DecisionTreeRegressor(max_depth=3, min_samples_leaf=5)
biden_tree.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=5,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [ ]:
biden_tree_viz = export_graphviz(
    biden_tree,
    out_file=None,
    feature_names=["female","age","educ","dem","rep"],
    class_names=data.biden,
    rounded=True,
    filled=True,
)

graph = graphviz.Source(biden_tree_viz)
graph.render('biden_tree_viz')
graph

In [ ]:
y_pred_1 = biden_tree.predict(X_test)
mse_1 = mean_squared_error(y_pred_1, y_test)
print('MSE = {}'.format(mse_1))

#### (b)

In [ ]:
param_dist1 = {'max_depth': [3, 10],
             'min_samples_split': sp_randint(2, 20),
             'min_samples_leaf': sp_randint(2, 20)}

In [ ]:
biden_tree_2 = DecisionTreeRegressor()

random_search1 = \
    RandomizedSearchCV(biden_tree_2, param_distributions = param_dist1,
                       n_iter=100, n_jobs=-1, cv=5, random_state=25,
                       scoring='neg_mean_squared_error')

In [ ]:
rs_fit1 = random_search1.fit(X_train, y_train)
print('RandBestEstimator1=', random_search1.best_estimator_)
print('RandBestParams1=', random_search1.best_params_)
print('RandBestScore1=', -random_search1.best_score_)

#### (c)

In [ ]:
param_dist2 = {'n_estimators': [10, 200],
             'max_depth': [3, 10],
             'min_samples_split': sp_randint(2, 20),
             'min_samples_leaf': sp_randint(2, 20),
             'max_features': sp_randint(1, 5)}

In [ ]:
random_forest = RandomForestRegressor()
random_search2 = RandomizedSearchCV(random_forest, param_distributions=param_dist2,
                                   n_iter=100, n_jobs=-1, cv=5, random_state=25, scoring='neg_mean_squared_error')
rs_fit2 = random_search2.fit(X_train, y_train)

In [ ]:
print('RandBestEstimator2 = ', rs_fit2)
print('RandBestParams2 = ', rs_fit2.best_params_)
print("MSE = ", abs(rs_fit2.best_score_))

#### 2. Classifier “horse” race

#### (a)

In [ ]:
# load the data
df = pd.read_csv('Auto.csv', na_values='?')
df.columns=['mpg', 'cyl', 'displ', 'hpwr', 'wgt', 'accl', 'yr', 'orgn','name']
df['mpg_high'] = (df['mpg']>=df['mpg'].median()).astype('int')
df.dropna(inplace=True)
df_orgn = pd.get_dummies(df.orgn, prefix='orgn').iloc[:, :-1]
df = pd.concat([df, df_orgn], axis=1)
df.head()

In [ ]:
k = 4
kf = KFold(k, shuffle=True, random_state=25)
Xvars = df[['cyl', 'displ', 'hpwr', 'wgt', 'accl', 'yr', 'orgn_1', 'orgn_2']].values
yvars = df.mpg_high.values
kf.get_n_splits(Xvars)
MSE_vec_kf = np.zeros(k)
y_test_lst2 = np.zeros(df.shape[0])
y_pred_lst2 = np.zeros(df.shape[0])

In [ ]:
k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvars[train_index], yvars[test_index]
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    y_test_lst2[test_index] = y_test
    y_pred_lst2[test_index] = y_pred
    MSE_vec_kf[k_ind] = (y_test != y_pred).mean()
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
print('Test estimate MSE k-fold = {}.'.format(MSE_kf))
print(classification_report(y_test_lst2, y_pred_lst2))


 The error rate for mpg_high=0 is 0.08. The error rate for mpg_high=1 is 0.11.

#### (b)

In [ ]:
param_dist3 = {'n_estimators': [10, 200],
             'max_depth': [3, 8],
             'min_samples_split': sp_randint(2, 20),
             'min_samples_leaf': sp_randint(2, 20),
             'max_features': sp_randint(1, 8)}

In [ ]:
random_forest2 = RandomForestClassifier()
param_dist3 = {'n_estimators': [10, 200],
                'max_depth': [3, 8],
                'min_samples_split': sp_randint(2, 20),
                'min_samples_leaf': sp_randint(2, 20),
                'max_features': sp_randint(1, 8)}
random_search3 = RandomizedSearchCV(random_forest2, param_distributions=param_dist3,
                                   n_iter=100, n_jobs=-1, cv=4, random_state=25, scoring='neg_mean_squared_error')
rs_fit3 = random_search3.fit(Xvars, yvars)

In [ ]:
print('RandBestEstimator3 = ', rs_fit3)
print('RandBestParams3 = ', rs_fit3.best_params_)
print('MSE = ', abs(rs_fit3.best_score_))

#### (c)

In [ ]:
param_dist4 = {'C': sp_uniform(loc=0.2, scale=4.0),
                'gamma': ['scale', 'auto'],
               'shrinking': [True, False]}

In [ ]:
svc = SVC(kernel='rbf')
random_search4 = RandomizedSearchCV(svc, param_distributions=param_dist4, n_iter=100, n_jobs=-1, cv=4, random_state=25, scoring='neg_mean_squared_error')
rs_fit4 = random_search4.fit(Xvars, yvars)

In [ ]:
print('RandBestEstimator3 = ', rs_fit4)
print('RandBestParams3 = ', rs_fit4.best_params_)
print('MSE = ', abs(rs_fit4.best_score_))

#### (d)

The random forest predictor is the best predictor for mpg_high in terms of MSE.